In [118]:
import re
import os
import numpy as np
import pandas as pd
import itertools
from __future__ import print_function, division
from IPython.display import display

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1)
matplotlib.style.use('ggplot')
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

In [83]:
# match this pattern G[E/D]R[L/M]R[V/F]T
m = ['G',['E','D'],'R',['L','M'],'R',['V','F'],'T']
mp = [['E','D'],['L','M'],['V','F']]

motifs = []
for i in itertools.product(mp[0],mp[1],mp[2]):
    motifs.append(''.join([m[0],i[0],m[2],i[1],m[4],i[2],m[6]]))

# write out a file of target sequences
!rm blast_targets.fasta 
!touch blast_targets.fasta 
for i,m in enumerate(motifs):
    ! echo '>target'{i} >> blast_targets.fasta 
    ! echo {m} >> blast_targets.fasta 

## Fetch pBLAST against Agrobacterium

In [75]:
# TODO: Get query against NCBIWWW to work instead of manually downloading the files
# from Bio.Blast import NCBIWWW 
# result_handle = NCBIWWW.qblast("blastp", "Microbial proteins from nr", motifs[0])
# blast_file = open("Blast_Results/"+motifs[0]+".xml", "w")  

# blast_file.write(result_handle.read()) 
# blast_file.close()# tidy up
# result_handle.close()

In [84]:
# Currently works based on first 100 results for each motif
# https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&PROG_DEFAULTS=on&PROG_DEF=blastp&BLAST_SPEC=MicrobialGenomes_1435057&DB_GROUP=AllMG

## Construct a dataframe of all the motifs

In [88]:
from Bio import SeqIO
from Bio.Alphabet import IUPAC

dataFrames = []

for motif in motifs:
    fasta_sequences = SeqIO.parse(open('Blast_Results/'+motif+'.fasta'),'fasta',alphabet=IUPAC.protein)
    for fasta in fasta_sequences:
        row = [motif,fasta.id, fasta.description, fasta.seq.tostring()]
        df = pd.DataFrame(data=[row],columns=['motif','name', 'desc', 'seq'])
        dataFrames.append(df)

df = pd.concat(dataFrames,ignore_index=True)
df.sample(10)

,motif,name,desc,seq
139,GERLRFT,WP_003516634.1,WP_003516634.1 MULTISPECIES: aconitate hydrata...,MPKSLDSFHCRSVLTVDGKDYVYFSLPKAEANGLKGVSKLPYSMKV...
722,GDRMRFT,AHK00242.1,AHK00242.1 DNA mismatch repair protein MutS [A...,MEQYIEIKANNPGSLLFYRMGDFYELFFDDAVEASRSLGITLTKRG...
657,GDRMRVT,AHK03178.1,AHK03178.1 maltose operon transcriptional repr...,MAENMKLKEFAEKVGLSPTTVSRALGGYPEVREETRQRVMDAALKY...
761,GDRMRFT,WP_038492348.1,WP_038492348.1 MULTISPECIES: polysaccharide bi...,MQSLIKPQNLTATLMGKTGDTLLWLLAALHVTAVAILFATLLSLGE...
265,GERMRVT,WP_038493908.1,WP_038493908.1 MULTISPECIES: peptide ABC trans...,MSFTLRLLRSFEGAAGAIILTLLAVTALAAPLLFPGDPLSIVGEPL...
292,GERMRVT,AHK01130.1,AHK01130.1 ATP-dependent protease La [Agrobact...,MKGNDMTNITSAASGGIYPVLPLRDIVVFPHMIVPLFVGREKSIRA...
399,GERMRFT,WP_003515403.1,WP_003515403.1 MULTISPECIES: 3-deoxy-7-phospho...,MAQNWTPGSWRQKPIQQVPEYPDAAALAATEATLATYPPLVFAGEA...
681,GDRMRVT,WP_003516002.1,WP_003516002.1 MULTISPECIES: phosphoribosylami...,MNRRRRIYEGKAKILYEGPEPGTLIQFFKDDATAFNKKKHEVIDGK...
628,GDRMRVT,WP_038497356.1,WP_038497356.1 MULTISPECIES: type I secretion ...,MPLSDMTTEQWPDPNAAGTDFASWSEALQYVARHYGVPFSPGGAQQ...
184,GERLRFT,WP_003513631.1,WP_003513631.1 MULTISPECIES: ribosome biogenes...,MSFTVAIVGRPNVGKSTLFNRLVGKKLALVDDTPGVTRDRRPGDAK...


In [128]:
def matchMotif(m,seq):
    regex = '('+m[0]+'?'+m[1]+'?'+m[2:4]+m[5]+'?'+m[6]+'?)' # (G?E?RMRF?T?)
    matches = re.finditer(regex, seq)
    results = [(m.start(0)/len(seq),m.group(0)) for m in matches]
    return results

df['matches'] = df.apply(lambda x: matchMotif(x.motif,x.seq),axis=1)
df.sample(10)

,motif,name,desc,seq,matches
144,GERLRFT,WP_003511299.1,WP_003511299.1 MULTISPECIES: sugar ABC transpo...,MADMTQSNSTASPKTEKTGSISRFISATELDTRLLGMVGALLLIWI...,"[(0.0732265446224, RL), (0.418764302059, RL), ..."
185,GERLRFT,WP_038497034.1,WP_038497034.1 MULTISPECIES: hypothetical prot...,MLEPSMPLARVKQIVAVIVLAFSAILIFLAIDGLTMNMRLTLAVFV...,"[(0.0806794055202, RLT), (0.242038216561, RL),..."
346,GERMRFT,WP_025594159.1,WP_025594159.1 MULTISPECIES: hypothetical prot...,MLPACRGLGKNEKQIRPFWQNYGSFVCFALAKSVNILRSNRMTRFL...,"[(0.493827160494, RMT), (0.962962962963, RM)]"
189,GERLRFT,WP_038494044.1,WP_038494044.1 MULTISPECIES: Fis family transc...,MMAGGTIFLVDDDSQLRKAMVQTLELDGLPVTSFSRAEQALAALNE...,"[(0.270925110132, RL), (0.334801762115, ERL), ..."
212,GERMRVT,WP_003511019.1,WP_003511019.1 MULTISPECIES: peptide ABC trans...,MTEPLLKVENLNVTFTNYGRTRQVLRDVAFTVSAGERVALIGETGS...,"[(0.600364963504, RM), (0.71897810219, RM), (0..."
103,GERLRFT,AHK01725.1,AHK01725.1 alanyl-tRNA synthetase [Agrobacteri...,MGMSGVNEIRSTFLDYFKKNGHEIVPSSPLVPRNDPTLMFTNAGMV...,"[(0.140607424072, RL), (0.629921259843, RL), (..."
372,GERMRFT,WP_046205506.1,WP_046205506.1 MULTISPECIES: phosphonate ABC t...,MATTLQHGTLTPEGLSTSSRTVMRHYQQQLRTRRIYTVMSLVIFLI...,[]
397,GERMRFT,WP_019564934.1,WP_019564934.1 MULTISPECIES: L-asparagine perm...,MKSVKTAPAEREVFTEEDLGYHKALKPRQIQMIAIGGAIGTGLFLG...,"[(0.857731958763, RMF)]"
314,GERMRFT,WP_003511245.1,WP_003511245.1 MULTISPECIES: hypothetical prot...,MSSDPKNPDFQDPRIAALMASTERDRPVADSSRRVGRDGREFCIYP...,"[(0.238770685579, RM), (0.418439716312, RMF), ..."
42,GERLRVT,AHK01725.1,AHK01725.1 alanyl-tRNA synthetase [Agrobacteri...,MGMSGVNEIRSTFLDYFKKNGHEIVPSSPLVPRNDPTLMFTNAGMV...,"[(0.140607424072, RL), (0.629921259843, RL), (..."


In [129]:
df.matches

0         [(0.585185185185, RLT), (0.908641975309, ERL)]
1      [(0.262411347518, ERL), (0.333333333333, ERL),...
2                                 [(0.408284023669, RL)]
3           [(0.505843071786, RL), (0.848080133556, RL)]
4      [(0.00102669404517, RL), (0.041067761807, RL),...
5         [(0.364864864865, RL), (0.445945945946, GERL)]
6         [(0.354901960784, RL), (0.437254901961, GERL)]
7      [(0.0756097560976, ERLV), (0.124390243902, RL)...
8      [(0.141089108911, RL), (0.683168316832, GRL), ...
9      [(0.324742268041, GERL), (0.54381443299, RL), ...
10     [(0.124675324675, GRL), (0.145454545455, GERL)...
11     [(0.115485564304, GRL), (0.136482939633, GERL)...
12     [(0.403174603175, GERL), (0.685714285714, RLV)...
13     [(0.0754098360656, RL), (0.501639344262, GERL)...
14     [(0.0693069306931, RL), (0.498349834983, GERL)...
15     [(0.0469314079422, RL), (0.29963898917, GERL),...
16        [(0.399209486166, GERL), (0.509881422925, RL)]
17     [(0.0207468879668, RL), 